<a href="https://colab.research.google.com/github/chaudhreysab11/XLM-Roberta-Base-Model/blob/main/XLM_Roberta_Base_POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparing Data

First, let's import the necessary Python modules.
In google colab you will need to install the some libraries.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets accelerate seqeval torch
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=c60efa4184d24329bf723dc357a80085ff9f84f789b0a3fedc9b9fd889705c66
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np
from datasets import load_metric,Dataset,DatasetDict,ClassLabel

In [5]:
model_checkpoint = 'xlm-roberta-base'
batch_size = 16

In [6]:
infile = open('/content/drive/MyDrive/FYP/Training_data.txt', 'r', encoding='utf-8-sig')
train = infile.readlines()
infile.close()

words = []
tags = []
for i in train:
    data = i.split("\n")[0]
    data = data.split("\t")
    words.append(data[0])
    tags.append(data[1])


In [7]:
vocabulary = []
for tag in tags:
  if tag not in vocabulary:
    vocabulary.append(tag)
vocab = {}
for i in range(len(vocabulary)):
  vocab[i] = vocabulary[i]
label_encoder = ClassLabel(names=vocabulary)
tags = label_encoder.str2int(tags)

In [8]:
sentences = []
pos_tags = []
index = 0
for i in range(len(words)):
  if words[i]=='۔':
    sentences.append(words[index:i+1])
    pos_tags.append(tags[index:i+1])
    index = i+1

In [9]:
tokenizer = AutoTokenizer.from_pretrained("facebook/xlm-roberta-xl")

In [10]:
init_token = tokenizer.cls_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, pad_token, unk_token)

<s> <pad> <unk>


In [11]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, pad_token_idx, unk_token_idx)

0 1 3


In [14]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    sentences,
    pos_tags,
    test_size = 0.4
)

In [15]:
pos_train_dataset = Dataset.from_dict({'sentence':x_train,'tags':y_train})
pos_val_dataset = Dataset.from_dict({'sentence':x_val[:int(len(x_val)*0.5)],'tags':y_val[:int(len(y_val)*0.5)]})
pos_test_dataset = Dataset.from_dict({'sentence':x_val[int(len(x_val)*0.5):],'tags':y_val[int(len(y_val)*0.5):]})
dataset = DatasetDict({'train':pos_train_dataset,'validation':pos_val_dataset,'test':pos_test_dataset})


In [16]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [17]:
show_random_elements(dataset["train"])

,sentence,tags
0,"[اراکین, کو, اولین, مراسلے, میں, تصویر, نظر, نہیں, آ, کے, لئے, بکرے, نے, انھیں, کتنا, پریشان, ۔]","[0, 2, 19, 0, 2, 0, 0, 16, 5, 2, 2, 1, 2, 18, 6, 1, 4]"
1,"[کیوں, ڈر, نہ, ہو, ظلمت, کو, مجھ, سے, ؟, شریعت, نافذ, نہیں, ،, پرانا, سوات, کا, قانون, نافذ, جسے, شریعت, کا, لبادہ, حکومت, اور, طالبان, دونوں, چین, کی, بجانے, کا, ۔]","[26, 0, 16, 5, 0, 2, 18, 2, 4, 0, 0, 16, 4, 1, 10, 2, 0, 0, 3, 0, 2, 0, 0, 9, 0, 19, 10, 2, 11, 2, 4]"
2,"[بہت, خوب, فرخ, صاحب, ارشاد, کا, کہنا, ’, متحدہ, کے, سربراہ, الطاف, دو, ہزار, دو, سے, اعلانیہ, بنے, اور, انہوں, نے, صدر, مشرف, کے, ہونے, کا, بیان, کے, سترہ, دن, بعد, سندھ, کا, استعمال, شروع, ۔]","[6, 1, 0, 0, 0, 2, 11, 4, 1, 2, 0, 10, 8, 8, 8, 2, 26, 5, 9, 18, 2, 0, 10, 2, 11, 2, 0, 2, 8, 0, 0, 10, 2, 0, 0, 4]"
3,"[!, دو, ،, تین, دن, سے, قدر, افطاری, ہورہی, کہ, کھانا, برائے, نام, کھاتا, ،, بلکہ, کھاتا, ،, ۔]","[4, 8, 4, 8, 0, 2, 0, 30, 0, 25, 0, 26, 0, 5, 4, 9, 5, 4, 4]"
4,"[پہلا, شعر, شاعر, کے, مقصد, کو, پیش, کرتا, سے, نجی, ٹی, وی, چینلز, اور, کے, مخصوص, پروگراموں, کی, مقبولیت, کا, تعین, جاسکے, گا, ۔]","[19, 0, 0, 2, 0, 2, 0, 5, 2, 1, 0, 0, 0, 9, 2, 1, 0, 2, 0, 2, 0, 0, 7, 4]"
5,"[میرے, سامنے, ایک, سرکاری, عمارت, سے, پاکستان, کا, پرچم, اتار, گیا, اور, میں, کچھ, نہ, سکا, ۔]","[29, 0, 8, 1, 0, 2, 10, 2, 0, 5, 15, 9, 2, 6, 16, 12, 4]"
6,"[کا, ڈیزائن, اور, سائز, بدلتا, رنگ, کبھی, نہ, بدلا, ۔]","[2, 0, 9, 0, 5, 0, 0, 16, 5, 4]"
7,"[بولا, ۔]","[5, 4]"
8,"[پھر, بھی, ،, پیرس, نے, اس, علاقہ, کی, طرف, کوئی, توجہ, نہیں, دی, جس, سے, حالت, بد, سے, بدتر, ہوتی, گئی, ۔]","[24, 2, 4, 10, 2, 14, 0, 2, 0, 14, 0, 16, 5, 3, 2, 0, 0, 2, 0, 5, 15, 4]"
9,"[فاتح, بھائی, اپنی, عقل, کو, "", چھوٹی, عقل, "", کہہ, ۔]","[1, 0, 21, 0, 2, 9, 1, 0, 9, 5, 4]"


# Preprocessing Data

In [18]:
example = dataset["train"][1]
print(example["sentence"])

['محی', 'الدین', 'قادری', 'کا', 'تعارف', 'کچھ', 'یوں', 'کہ', 'پاکستان', 'کی', 'مشہور', 'و', 'معروف', 'شخصیت', 'پروفیسر', 'ڈاکٹر', 'محمد', 'طاہر', 'القادری', 'صاحب', 'کے', 'چھوٹے', 'بیٹے', '۔']


In [20]:
tokenized_input = tokenizer(example["sentence"],is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '▁مح', 'ی', '▁الدین', '▁قادر', 'ی', '▁کا', '▁', 'تعارف', '▁کچھ', '▁یوں', '▁کہ', '▁پاکستان', '▁کی', '▁مشہور', '▁و', '▁معروف', '▁شخصیت', '▁پروفیسر', '▁ڈاکٹر', '▁محمد', '▁طاہر', '▁القادر', 'ی', '▁صاحب', '▁کے', '▁چھوٹے', '▁بیٹے', '▁۔', '</s>']


In [21]:
len(example[f"tags"]), len(tokenized_input["input_ids"])

(24, 30)

In [22]:
print(tokenized_input.word_ids())

[None, 0, 0, 1, 2, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 23, None]


In [23]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

30 30


In [24]:
label_all_tokens = True

In [25]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["sentence"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [26]:
tokenize_and_align_labels(dataset['validation'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[0, 12737, 5503, 754, 554, 25945, 140, 288, 16413, 38166, 504, 1568, 25945, 2185, 1262, 50072, 1578, 8880, 715, 19277, 52988, 199913, 490, 165706, 8641, 42318, 725, 1241, 716, 216, 4199, 754, 19573, 6, 142440, 639, 715, 37889, 230, 69294, 4647, 2437, 7482, 725, 12737, 40110, 4145, 21117, 746, 64482, 554, 24871, 148772, 488, 4351, 216, 11690, 317, 230, 8258, 11967, 16834, 639, 9446, 1999, 1423, 5818, 113768, 904, 504, 47068, 113768, 1923, 639, 9446, 5106, 15947, 1938, 639, 924, 1423, 368, 81400, 140, 5650, 499, 2598, 153118, 70026, 19486, 6067, 715, 9748, 187689, 218321, 288, 156197, 715, 8598, 216, 183790, 15, 3202, 2727, 1388, 504, 6708, 26333, 5023, 12737, 53911, 99829, 216, 554, 907, 488, 240417, 504, 216358, 5650, 6, 218321, 778, 6708, 26333, 490, 37889, 5461, 488, 14675, 1069, 504, 84180, 5023, 490, 715, 6514, 8640, 16549, 196043, 5028, 554, 26202, 3138, 5393, 70008, 3210, 7314, 21949, 201133, 504, 6708, 26333, 2047, 12288, 639, 725, 51378, 924, 3067, 639, 924, 1423

In [27]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/25326 [00:00<?, ? examples/s]

Map:   0%|          | 0/8442 [00:00<?, ? examples/s]

Map:   0%|          | 0/8443 [00:00<?, ? examples/s]

In [28]:
tokenized_datasets["train"].features[f"tags"].feature = label_encoder
tokenized_datasets["validation"].features[f"tags"].feature = label_encoder
tokenized_datasets["test"].features[f"tags"].feature = label_encoder

In [29]:
label_list = label_encoder.names
label_list

['NN',
 'JJ',
 'PSP',
 'PRR',
 'PU',
 'VBF',
 'Q',
 'AUXT',
 'CD',
 'CC',
 'NNP',
 'VBI',
 'AUXM',
 'VALA',
 'PDM',
 'AUXA',
 'NEG',
 'PRF',
 'PRP',
 'OD',
 'SCK',
 'PPR',
 'AUXP',
 'PRD',
 'SCP',
 'SC',
 'RB',
 'INJ',
 'PRE',
 'PRS',
 'FR',
 'SYM',
 'QM',
 'PRT',
 'FF',
 'AUXT ',
 'PSP ',
 'VBF ',
 '',
 'PU’']

In [30]:
metric = load_metric("seqeval")

<ipython-input-30-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [31]:
labels = [label_list[i] for i in example[f"tags"]]

metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: JJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Q seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: RB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarn

{'B': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'C': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'J': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 5},
 'N': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 5},
 'NP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'SP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'U': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 '_': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [32]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [33]:
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label,
                                                        label2id=label2id, num_labels=len(label_list))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [35]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-pos",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [36]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 277,483,816 trainable parameters


In [37]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/muhammadbilal/xlm-roberta-base-finetuned-pos into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/1.03G [00:00<?, ?B/s]

Download file runs/May21_20-37-08_62c4735702cf/1684701766.886929/events.out.tfevents.1684701766.62c4735702cf.1…

Download file runs/May21_20-37-08_62c4735702cf/events.out.tfevents.1684706632.62c4735702cf.1005.2: 100%|######…

Download file runs/May21_20-37-08_62c4735702cf/events.out.tfevents.1684701766.62c4735702cf.1005.0: 100%|######…

...

Clean file runs/May21_20-37-08_62c4735702cf/events.out.tfevents.1684706632.62c4735702cf.1005.2: 100%|#########…

Clean file training_args.bin:  28%|##8       | 1.00k/3.56k [00:00<?, ?B/s]

Download file tokenizer.json:   0%|          | 17.4k/16.3M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/16.3M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.03G [00:00<?, ?B/s]

In [41]:
!rm -rf /content/xlm-roberta-base-finetuned-pos/runs

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.093200,0.081158,0.973668,0.973875,0.973772,0.975855
2,0.059300,0.075355,0.976161,0.978323,0.977240,0.979204


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SCP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VALA seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: Use

In [ ]:
trainer.evaluate()

{'eval_loss': 0.06829696148633957,
 'eval_precision': 0.9799975532175189,
 'eval_recall': 0.981859410430839,
 'eval_f1': 0.9809275983468544,
 'eval_accuracy': 0.9821812623709922,
 'eval_runtime': 50.5578,
 'eval_samples_per_second': 166.977,
 'eval_steps_per_second': 10.443,
 'epoch': 5.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'ALA': {'precision': 0.9960732984293194,
  'recall': 0.9960732984293194,
  'f1': 0.9960732984293194,
  'number': 764},
 'B': {'precision': 0.9717691342534505,
  'recall': 0.9693366708385481,
  'f1': 0.9705513784461153,
  'number': 1598},
 'BF': {'precision': 0.9722851664633083,
  'recall': 0.9753453400944221,
  'f1': 0.9738128491620112,
  'number': 11438},
 'BI': {'precision': 0.9817336433078712,
  'recall': 0.98369384359401,
  'f1': 0.9827127659574469,
  'number': 3005},
 'C': {'precision': 0.9955647240845796,
  'recall': 0.9951541396020208,
  'f1': 0.9953593895019077,
  'number': 9699},
 'CK': {'precision': 0.9679144385026738,
  'recall': 0.9945054945054945,
  'f1': 0.9810298102981031,
  'number': 182},
 'CP': {'precision': 0.875,
  'recall': 0.8561151079136691,
  'f1': 0.8654545454545455,
  'number': 139},
 'D': {'precision': 0.9776242184929254,
  'recall': 0.9756978653530378,
  'f1': 0.9766600920447075,
  'number': 3045},
 'DM': {'precision': 0.9399293286219081,
  'recall': 0.9693

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.03G [00:00<?, ?B/s]

Upload file runs/May21_20-37-08_62c4735702cf/events.out.tfevents.1684701766.62c4735702cf.1005.0:   0%|        …

Upload file runs/May21_20-37-08_62c4735702cf/events.out.tfevents.1684706632.62c4735702cf.1005.2:   0%|        …

To https://huggingface.co/muhammadbilal/xlm-roberta-base-finetuned-pos
   f988d60..997cde7  main -> main

   f988d60..997cde7  main -> main

To https://huggingface.co/muhammadbilal/xlm-roberta-base-finetuned-pos
   997cde7..7253409  main -> main

   997cde7..7253409  main -> main



'https://huggingface.co/muhammadbilal/xlm-roberta-base-finetuned-pos/commit/997cde73349e653951451798ec1e6a4df13f754b'